In [2]:
# Add lxmls path to root.
import sys
import os
base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(os.path.dirname('__file__')))))
base_path = '/home/mariana.almeida/LxMLS/git/lxmls-toolkit'
sys.path.append(base_path)

In [3]:
# Other imports.
import sys
import lxmls.readers.simple_sequence as ssr
import lxmls.sequences.hmm as hmmc
import lxmls.readers.pos_corpus as pcc
import lxmls.sequences.confusion_matrix as cm


# Exercise 2.1
Load the simple sequence dataset. From the ipython command line create a simple sequence object and look at the training and test set.

In [4]:
simple = ssr.SimpleSequence()
print simple.train
print simple.test

[walk/rainy walk/sunny shop/sunny clean/sunny , walk/rainy walk/rainy shop/rainy clean/sunny , walk/sunny shop/sunny shop/sunny clean/sunny ]
[walk/rainy walk/sunny shop/sunny clean/sunny , clean/sunny walk/sunny tennis/sunny walk/sunny ]


In [5]:
for sequence in simple.train.seq_list:
    print sequence

walk/rainy walk/sunny shop/sunny clean/sunny 
walk/rainy walk/rainy shop/rainy clean/sunny 
walk/sunny shop/sunny shop/sunny clean/sunny 


In [6]:
for sequence in simple.train.seq_list:
    print sequence.x

[0, 0, 1, 2]
[0, 0, 1, 2]
[0, 1, 1, 2]


In [7]:
for sequence in simple.train.seq_list:
    print sequence.y

[0, 1, 1, 1]
[0, 0, 0, 1]
[1, 1, 1, 1]


# Exercise 2.2
The provided function train supervised from the hmm.py file implements the above parameter estimates.
Run this function given the simple dataset above and look at the estimated probabilities. Are they correct? You can also
check the variables ending in counts instead of probs to see the raw counts (for example, typing hmm.initial counts
will show you the raw counts of initial states). How are the counts related to the probabilities?

In [9]:
hmm = hmmc.HMM(simple.x_dict, simple.y_dict)
hmm.train_supervised(simple.train)

print "Initial Probabilities:"
print hmm.initial_probs

Initial Probabilities:
[0.66666667 0.33333333]


In [10]:
print "Transition Probabilities:"
print hmm.transition_probs

Transition Probabilities:
[[0.5   0.   ]
 [0.5   0.625]]


In [11]:
print "Final Probabilities:"
print hmm.final_probs

Final Probabilities:
[0.    0.375]


In [12]:
print "Emission Probabilities"
print hmm.emission_probs

Emission Probabilities
[[0.75  0.25 ]
 [0.25  0.375]
 [0.    0.375]
 [0.    0.   ]]


# Exercise 2.3 
Convince yourself that the score of a path in the trellis (summing over the scores above) is equivalent to the
log-probability $\log P(X = x, Y = y)$, as defined in Eq. 2.2. Use the given function compute scores on the first training
sequence and confirm that the values are correct. You should get the same values as presented below

In [15]:
initial_scores, transition_scores, final_scores, emission_scores = \
    hmm.compute_scores(simple.train.seq_list[0])
print initial_scores

[-0.40546511 -1.09861229]


In [16]:
print transition_scores

[[[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]

 [[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]

 [[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]]


In [17]:
print final_scores

[       -inf -0.98082925]


In [18]:
print emission_scores

[[-0.28768207 -1.38629436]
 [-0.28768207 -1.38629436]
 [-1.38629436 -0.98082925]
 [       -inf -0.98082925]]


# Exercise 2.4 
Look at the module ``sequences/log domain.py``. This module implements a function ``logsum pair(logx, logy)`` to add two numbers represented in the log-domain; it returns their sum also represented in the log-domain. The
function logsum(logv) sums all components of an array represented in the log-domain. This will be used later in our
decoding algorithms. To observe why this is important, type the following:

In [20]:
import numpy as np

a = np.random.rand(10)
print np.log(sum(np.exp(a)))
print np.log(sum(np.exp(10*a)))
print np.log(sum(np.exp(100*a)))
print np.log(sum(np.exp(1000*a)))

2.87565803223805
10.757717497406828
97.95088683170002
inf


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


In [21]:
from lxmls.sequences.log_domain import *

print logsum(a)
print logsum(10*a)
print logsum(100*a)
print logsum(1000*a)

2.8756580322380505
10.757717497406826
97.95088683170003
978.9184679696508


# Exercise 2.5 
Run the provided forward-backward algorithm on the first train sequence. Observe that both the forward
and the backward passes give the same log-likelihood.

In [23]:
log_likelihood, forward = hmm.decoder.run_forward(initial_scores, transition_scores, final_scores, emission_scores)
print 'Log-Likelihood =', log_likelihood

Log-Likelihood = -5.068232326005127


In [24]:
log_likelihood, backward = hmm.decoder.run_backward(initial_scores, transition_scores, final_scores, emission_scores)
print 'Log-Likelihood =', log_likelihood

Log-Likelihood = -5.068232326005126


# Exercise 2.6 
Compute the node posteriors for the first training sequence (use the provided compute posteriors function), and look at the output. Note that the state posteriors are a proper probability distribution (the lines of the result
sum to 1)

In [25]:
initial_scores, transition_scores, final_scores, emission_scores = \
    hmm.compute_scores(simple.train.seq_list[0])
state_posteriors, _, _ = hmm.compute_posteriors(initial_scores,
                                                transition_scores,
                                                final_scores,
                                                emission_scores)
print state_posteriors

[[0.95738152 0.04261848]
 [0.75281282 0.24718718]
 [0.26184794 0.73815206]
 [0.         1.        ]]


# Exercise 2.7 
Run the posterior decode on the first test sequence, and evaluate it.

In [27]:
y_pred = hmm.posterior_decode(simple.test.seq_list[0])
print "Prediction test 0", y_pred
print "Truth test 0", simple.test.seq_list[0]

Prediction test 0 walk/rainy walk/rainy shop/sunny clean/sunny 
Truth test 0 walk/rainy walk/sunny shop/sunny clean/sunny 


Do the same for the second test sequence:

In [29]:
y_pred = hmm.posterior_decode(simple.test.seq_list[1])
print "Prediction test 1", y_pred
print "Truth test 1", simple.test.seq_list[1]

Prediction test 1 clean/rainy walk/rainy tennis/rainy walk/rainy 
Truth test 1 clean/sunny walk/sunny tennis/sunny walk/sunny 


What is wrong? Note the observations for the second test sequence: the observation tennis was never seen at
training time, so the probability for it will be zero (no matter what state). This will make all possible state sequences have
zero probability. As seen in the previous lecture, this is a problem with generative models, which can be corrected using
smoothing (among other options).

Change the train supervised method to add smoothing:

    def train_supervised(self,sequence_list, smoothing):

In [31]:
hmm.train_supervised(simple.train, smoothing=0.1)

y_pred = hmm.posterior_decode(simple.test.seq_list[0])
print "Prediction test 0 with smoothing"
print y_pred
print "Truth test 0"
print simple.test.seq_list[0]

Prediction test 0 with smoothing
walk/rainy walk/rainy shop/sunny clean/sunny 
Truth test 0
walk/rainy walk/sunny shop/sunny clean/sunny 


In [32]:
y_pred = hmm.posterior_decode(simple.test.seq_list[1])
print "Prediction test 1 with smoothing"
print y_pred
print "Truth test 1"
print simple.test.seq_list[1]

Prediction test 1 with smoothing
clean/sunny walk/sunny tennis/sunny walk/sunny 
Truth test 1
clean/sunny walk/sunny tennis/sunny walk/sunny 
